In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
data = None
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        data = pd.read_csv(os.path.join(dirname, filename))
data_values = data.values
#làm cho các giá trị trong dữ liệu dễ sử dụng hơn
def clean_data(data):
    data_copy = data.copy()
    max_col = np.max(data, axis = 0)
    min_col = np.min(data, axis = 0)
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            data_copy[i, j] = (data_copy[i, j] - min_col[j])/(max_col[j] - min_col[j])
    return data_copy
# sắp lại vị trí các bộdữ liệu để cho cân bằng target 1-0 
def balanced_data(data):
    new_data = data.copy()
    mid = int((data.shape[0] - (0 if data.shape[0]%2 == 0 else 1))/ 2)
    count = 0
    for i in range(mid):
        new_data[count] = data[i]
        new_data[count + 1] = data[i + mid]
        count += 2
    return new_data
data_values = clean_data(data_values)
data_values = balanced_data(data_values)
data_values = data_values.astype(np.float)
            
#print(data)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
print(data.values.shape[1])
#Số lượng kiểu thông tin của mỗi người khám và kết quả thực tế
print(data.values.shape[0])
#Số lượng bộ dữ liệu hay số người khám 
number_split = 200
Max_val = np.max(data_values[:, :-1], axis = 0)
# Chia ra thành dãy để test và train
number_split = int((3 * data_values.shape[0])/4)
X_train = data_values[:number_split, :-1]
Y_train = data_values[:number_split, -1].reshape(-1,1)
X_test = data_values[number_split:, :-1]
Y_test = data_values[number_split:, -1].reshape(-1,1)
weight = [np.full((X_train.shape[1], 1), 0.001)]
bias = 0.
#Số vòng lặp để train
train_loop = 300
import math
def sigmoid(Z):
    new_Z = []
    for i in Z:
        if i > 0:
            new_Z.append(1/(math.exp(-i) + 1))
        else:
            new_Z.append(math.exp(i)/(math.exp(i) + 1))
    return np.array(Z).reshape(-1,1)
end_loop = False
learning_rate = 0.00005

In [ ]:
for_end = 10**-8
#dùng SGD
for i in range(train_loop):
    random = np.random.permutation(X_train.shape[0])
    if i%10 == 0:
        Y_trainPredict=sigmoid(X_train.dot(weight[-1])+bias)
        Loss = -np.sum(Y_train.T.dot(np.log(Y_trainPredict))+(1.-Y_train.T).dot(np.log(1.-Y_trainPredict)))
        print(Loss)
        #print(Y_predict)
    for j in random:
        Xj = X_train[j]
        Yj = Y_train[j]
        Yj_predict = sigmoid(Xj.dot(weight[-1]) + bias)
        new_weight = weight[-1] - learning_rate * Xj.reshape(-1,1).dot(Yj_predict - Yj)
        if(np.linalg.norm(new_weight - weight[-1]) < for_end):
            end_loop = True
        else:
            weight.append(new_weight)
            bias -= learning_rate * np.sum(Yj_predict - Yj, axis = 0)
        if(end_loop):
            break
    if(end_loop):
        break


In [ ]:
Y_testPredict = sigmoid(X_test.dot(weight[-1]) + bias)
test_false = 0
for score in abs(Y_testPredict - Y_test).reshape(-1):
    if score >= 0.5:
        test_false += 1
Y_trainPredict = sigmoid(X_train.dot(weight[-1]) + bias)
train_false = 0
for score in abs(Y_trainPredict - Y_train).reshape(-1):
    if score >= 0.5:
        train_false += 1
print("Test score is {}".format((Y_test.shape[0] - test_false)/Y_test.shape[0]))
print("Train score is {}".format((Y_train.shape[0] - train_false)/Y_train.shape[0]))